In [ ]:
class SVR_PerLs:
    
    def __init__(self, C = 0.1, kernel = "linear", **kernel_param):
        import numpy as np
        from numpy.linalg import inv
        from sklearn.metrics.pairwise import pairwise_kernels
        from sklearn.utils import check_X_y, check_array
        self.C = C
        self.kernel = kernel
        self.pairwise_kernels = pairwise_kernels
        self.kernel_param = kernel_param
        self.check_X_y = check_X_y
        self.check_array = check_array
        self.inv = inv
        
    def fit(self, X, y):
        X, y = self.check_X_y(X, y)
        # hyperparameters
        C = self.C 
        
        kernel = self.kernel
        pairwise_kernels = self.pairwise_kernels
        inv = self.inv
        
        # omega + upsilon
        omega_ = pairwise_kernels(X, X, kernel, **self.kernel_param) + np.identity(y.size)*((y**2)/C)
        # ones vector
        onev = np.ones(y.shape).reshape(-1, 1)
        y = y.reshape(-1, 1)
        
        # solve for parameters
        A = np.linalg.pinv(np.block([[0, onev.T],[onev, omega_]]))
        B = np.concatenate((np.array([0]),y.reshape(-1)))
        sol =  A @ B
        
        b = sol[0]
        alpha = sol[1:]
        
        self.X = X
        self.alpha = alpha; self.b = b
        return self
        
    def predict(self, X_):
        pairwise_kernels = self.pairwise_kernels
        kernel_param = self.kernel_param
        kernel = self.kernel
        alpha = self.alpha
        b = self.b
        X = self.X
        
        X_ = self.check_array(X_)
        predict = alpha @ pairwise_kernels(X, X_, metric = kernel, **kernel_param) + b
        return predict
    
    
        # coefficient
    def coef_(self):
        if self.kernel == "linear":
            alpha = self.alpha; X = self.X
            w = alpha @ X
            return w, self.b
        else: 
            return self.alpha,  self.b, self.X